In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
)
import transformers

from datasets import load_dataset
from composer import Trainer    
from composer.models import HuggingFaceModel

from torch.utils.data import DataLoader

model_name = "roneneldan/TinyStories-1M"
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_config(config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model.tie_weights()
sum(p.numel() for p in model.parameters() if p.requires_grad)

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

3745984

In [2]:
train_data = load_dataset('roneneldan/TinyStories', split='train')
val_data = load_dataset('roneneldan/TinyStories', split='validation')

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [17]:
def tokenize(e):
    return tokenizer(e['text'], truncation=True, max_length=256, padding='max_length')
tokenizer.pad_token = tokenizer.eos_token
tok_train = train_data.select(range(100000)).map(tokenize, remove_columns=['text'])
tok_val  = val_data.select(range(1000)).map(tokenize, remove_columns=['text'])

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [19]:
collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
dl_train = DataLoader(tok_train, batch_size=8, shuffle=True, collate_fn=collator)
dl_val = DataLoader(tok_val, batch_size=8, shuffle=False, collate_fn=collator)

In [12]:
tokenizer

GPT2TokenizerFast(name_or_path='roneneldan/TinyStories-1M', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [14]:
next(iter(dl_train))['input_ids'].shape

torch.Size([8, 256])

In [18]:
hf_model = HuggingFaceModel(model=model, tokenizer=tokenizer)
trainer = Trainer(
    model=hf_model,
    train_dataloader=dl_train, 
    eval_dataloader=dl_val,
    max_duration="1ep",
    save_folder="training/pretrain/",
    save_interval="1ep",
    save_filename="hf_model.pt",
    save_overwrite=True,
)
trainer.fit()

/usr/lib/python3/dist-packages/composer/trainer/trainer.py:1258: UserWarning: No optimizer was specified. Defaulting to DecoupledSGDW(lr=0.1)
  warnings.warn((
******************************
Config:
composer_commit_hash: None
composer_version: 0.29.0
node_name: unknown because NODENAME environment variable not set
num_gpus_per_node: 1
num_nodes: 1
rank_zero_seed: 1818231930

******************************


train          Epoch   0:    0%|| 0/125 [00:00<?, ?ba/s]         

eval           Epoch   0:    0%|| 0/125 [00:00<?, ?ba/s]         

In [24]:
print(tokenizer.batch_decode(model.generate(**tokenizer("Once upon a time, in a land far away, there lived a", return_tensors='pt').to('cuda'), max_length=50, do_sample=True, top_k=50, top_p=0.95)))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Once upon a time, in a land far away, there lived a the could a help. He a his there,, time. She with. He but.\n\n She happy.\n\n\nThe girl to her to to with. He']


In [26]:
!git add .

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
!git status

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
